# Simple Dask example on a single machine

We define a simple function that simulates work by calling ```sleep(2)``` and call it multiple times. \
Since they are independent from each other, the Dask scheduler runs them in parallel.

In [ ]:
from dask.distributed import Client, LocalCluster

In [ ]:
from time import sleep

def foo(n):
    sleep(2)
    return n

There are many options to customize the creation of a local cluster (see [here](https://distributed.dask.org/en/latest/api.html#cluster)). \
In particular, ```processes=False``` (default) instructs the workers to use processes, while ```processes=True``` makes them use threads.

In [ ]:
# Start a local cluster and connect a client to it

cluster = LocalCluster()
client = Client(cluster)

In [ ]:
client

In [ ]:
%%time
# Sequential execution

results = [foo(n) for n in range(4)]

print('Results: {}'.format(results))

In [ ]:
%%time 
# Parallel execution

futures = client.map(foo, range(4))
results = client.gather(futures)
del futures

print('Results: {}'.format(results))

In [ ]:
%%time 
# This works as well

futures = [client.submit(foo, n) for n in range(4)]
results = client.gather(futures)
del futures

print('Results: {}'.format(results))

N.B.: ```map``` and ```submit``` return **immediately** ```future``` objects, which describe the status of the submitted computation but don't block the code. This means that we can let them run and fetch them as a later time using ```gather``` or ```result```.

In [ ]:
client.close()
cluster.close()